# Исследование надежности заемщиков.
# Research reliability borrowers

# Задача

<b>Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.<b>

### Шаг 1. Обзор данных
### Step 1. Data overview

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">

Данные содержат 21 525 строк и 12 столбцов. Все строки, кроме двух, не имеют пропущеных значений. 
    Название столбцов "правильные" и не требуют исправлений.

Строки `days_employed` и `total_income` имеют пропущенные значения и при этом имеют одинаковый тип данных `float64`. Необычным является и то что количество пропусков полность одинаково. Это наводит на мысль что возможно пропуски в одинаковых строках.
    
</div>

In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df.columns #получение списка с названиями столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
#просмотр уникальных значений столбцов, исключая столбцы с данными типа float64
list_columns = ['children', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
        'purpose']
for j in list_columns:
    print(j, sorted(df[j].unique()))
    print()

children [-1, 0, 1, 2, 3, 4, 5, 20]

dob_years [0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]

education ['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

education_id [0, 1, 2, 3, 4]

family_status ['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']

family_status_id [0, 1, 2, 3, 4]

gender ['F', 'M', 'XNA']

income_type ['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']

debt [0, 1]

purpose ['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться обр

<div class="alert alert-block alert-info">
    Просмотр показывает что встречаются некоторые ошибки в данных и странности, например:
    
    'children' - отрицательное количество детей, ненормально большое число детей 20
    'dob_years' - нулевой возраст человека
    'education' - одинаковое образование, записанное разными регистрами
    'education_id' - нормально
    'family_status' - нормально
    'family_status_id' - нормально, количество типов соответствует предыдущему столбцу
    'gender' - есть непонятный пол "XNA"
    'income_type' - нормально
    'debt' - нормально
    'purpose' - повторяющиеся значения написанные разными формулировками
        
</div>

### Шаг 2.1 Заполнение пропусков
### Step 2.1 Fill the pass

In [7]:
#дополнительная проверка на наличие пустых значений
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
#проверка на наличие полных дубликатов строк
df.duplicated().sum()

54

In [9]:
#избавимся от полных дубликатов и перезапишем датафрейм, со сбросом индексов и удалим столбец с индексами
df = df.drop_duplicates().reset_index(drop=True)

In [10]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21471 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">
количество строк, после удаления дубликатов, уменьшилось с 21525 до 21471
</div>

In [12]:
#доп проверка на наличие дубликатов строк
df.duplicated().sum()

0

In [13]:
#просмотр строк с пропущенными значениями
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


<div class="alert alert-block alert-info">
Пропущенные значения в столбцах days_employed и total_income типа NaN, команда tail() даёт тот же результат
</div>

In [14]:
#расчет процентов пропусков
100*(21471 - 19351) / 21471 #Полное кол-во значений 21471, кол-во значений в строках с пропусками 21471

9.873783242513158

<div class="alert alert-block alert-info">
Доля пропущенных значений практически 10%. Не сильно много, но и не мало. От таких строк лучше не избавляться (не удалять их), т.к. они повлияют на дальнейшие вычисления.
    
Пропуски находятся в столбцах "общий трудовой стаж в днях" и "ежемесячный доход". Причем пропуски синхронны, т.е. находятся в одной строке и принадлежат одному человеку. Возможно люди не заполнили эту информацию, потому что не работали официально.
    
Заполнить пропуски медианным значением лучше, потому что заработная плата у людей сильно отличается. В результате при заполнении средним значением у человека окажется з/п больше чем у большинства, что исказит результаты. Так же, с точки зрения теории вероятности, наиболее вероятно что человек имеет небольшую зарплату.
</div>

In [15]:
#расчет медианного значения по столбцу total_income
total_income_median = df['total_income'].median()
total_income_median

145017.93753253992

In [16]:
#заполнение столбца total_income медианным значением
df['total_income'] = df['total_income'].fillna(total_income_median)

In [17]:
#проверка 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  float64
 11  purpose           21471 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [18]:
df.loc[12] #проверка для 12-й строки. Данные в total_income действительно заменились

children                           0
days_employed                    NaN
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income           145017.937533
purpose              сыграть свадьбу
Name: 12, dtype: object

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [19]:
#Работа со столбцом days_employed - общий трудовой стаж в днях
df['days_employed'].sort_values()

16307   -18388.949901
4297    -17615.563266
7325    -16593.472817
17804   -16264.699501
16796   -16119.687737
             ...     
21435             NaN
21441             NaN
21443             NaN
21448             NaN
21456             NaN
Name: days_employed, Length: 21471, dtype: float64

In [20]:
# просмотр того же столбца с целью увидеть данные "по убыванию"
df['days_employed'].sort_values(ascending=False)

6950     401755.400475
9997     401715.811749
7660     401675.093434
2156     401674.466633
7790     401663.850046
             ...      
21435              NaN
21441              NaN
21443              NaN
21448              NaN
21456              NaN
Name: days_employed, Length: 21471, dtype: float64

<div class="alert alert-block alert-info">
Встречаются дробные положительные и отрицательные числа, а так же отсутствующие данные типа NaN
</div>

In [21]:
#количество отрицательных данных
df[df['days_employed'] < 0]['days_employed'].count()

15906

In [22]:
#мин значение в отрицательных данных
df[df['days_employed'] < 0]['days_employed'].min()

-18388.949900568383

In [23]:
#макс значение в отрицательных данных
df[df['days_employed'] < 0]['days_employed'].max()

-24.14163324048118

<div class="alert alert-block alert-info">
</div>

In [24]:
#количество положительных данных
df[df['days_employed'] > 0]['days_employed'].count()

3445

In [25]:
#мин значение в положительных данных
df[df['days_employed'] > 0]['days_employed'].min()

328728.72060451825

In [26]:
#макс значение в положительных данных
df[df['days_employed'] > 0]['days_employed'].max()

401755.40047533

<div class="alert alert-block alert-info">
</div>

In [27]:
#количество пустых данных
df['days_employed'].isna().sum()

2120

In [28]:
#количество нулевых данных
df[df['days_employed'] == 0]['days_employed'].count()

0

<div class="alert alert-block alert-info">
Отрицательных данных намного больше чем положительных. При этом положительных и отсутствующих данных примерно поровну (во всяком случае они одного порядка)
    
Не похоже что "минусы" появились при заполнении. Не могло столько людей ошибиться и написать отрицательный трудовой стаж. Так же большинство людей не знает сколько дней они работали, но многие знают сколько лет, но при этом данные дробные. Похоже что это связано с расчетом машинным методом.   

    
    В году примерно 247 рабочих дней.
    
Если посчитать стаж в годах, то для "положительных данных" получается нереальное число, около 1400 лет.

Для отрицательных данных, числа более вменяемые, максимум 72 года.
    
Заменим "положительные" данные медианным значением "отрицательных" данных
</div>

In [29]:
#расчет медианного значения по столбцу days_employed. 
#В расчёте участвуют только отрицательные данные, как более достоверные
days_employed_median = df[df['days_employed'] < 0 ]['days_employed'].median()

In [30]:
days_employed_median

-1630.0193809778218

In [31]:
#замена NaN на медианное значение
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

In [32]:
df['days_employed'] #длина стала 21471, значения NaN исчезли

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21466     -4529.316663
21467    343937.404131
21468     -2113.346888
21469     -3112.481705
21470     -1984.507589
Name: days_employed, Length: 21471, dtype: float64

In [33]:
#замена положительных значений на медианное отрицательное
df.loc[df['days_employed'] > 0, 'days_employed'] = days_employed_median

In [34]:
#изменение знака с минуса на плюс для столбца days_employed
df['days_employed'] = df['days_employed'].abs()

In [35]:
#проверка изменений
df['days_employed'].head()

0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
4    1630.019381
Name: days_employed, dtype: float64

<div class="alert alert-block alert-info">
по идее, трудовой стаж необходимо перевести в целочисленный тип. Это уменьшит занимаемое данными место и кроме того, не может стаж быть дробным числом.
    
</div>

### Шаг 2.3. Изменение типов данных.
### Step 2.3. Change data type.

In [36]:
#замена данных с вещественного типа на int в столбце total_income 
df['total_income'] = df['total_income'].astype('int')

In [37]:
#проверка
df['total_income']

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21466    224791
21467    155999
21468     89672
21469    244093
21470     82047
Name: total_income, Length: 21471, dtype: int64

### Шаг 2.4. Удаление дубликатов.
### Step 2.4. Delete of  duplicates.

In [38]:
#обработка неявных дубликатов. Неявные дубликаты присутствуют в двух столбцах education, purpose

In [39]:
#функция для замены неявных дубликатов
def replace_duplicates(wrong_values, correct_value, column):
    try:
        for wrong_value in wrong_values: # перебираем неверные данные
            df[column] = df[column].replace(wrong_value, correct_value) # заменяем неверные данные на верные
    except:
        print('Нет такого столбца!')

<div class="alert alert-block alert-info">
    
Замена дубликатов в столбце `education`
</div>

In [40]:
#все буквы в нижний регистр
df['education'] = df['education'].str.lower()

In [41]:
#проверка
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

<div class="alert alert-block alert-info">
    
Замена дубликатов в столбце `purpose`
</div>

In [42]:
#проверка
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [43]:
#функция для замены неявных дубликатов НОВАЯ_ВЕРСИЯ v1
def replace_dubl2(purpose):
    if  'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'

In [44]:
df['purpose_category'] = df['purpose'].apply(replace_dubl2)

In [45]:
#проверка
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

<div class="alert alert-block alert-info">

<b>Обработка аномальных значений<b>

    
Столбец `children`
    
</div>

In [46]:
#обработка столбца children, подсчёт количества записей с "отрицательным" ребёнком
df[df['children'] == -1]['children'].count()

47

In [47]:
#подсчёт количества записей с 20-ю детьми (интересно кол-во людей)
df[df['children'] == 20]['children'].count()

76

In [48]:
df['children'].mean() #среднее кол-во детей

0.5395649946439384

In [49]:
df['children'].median() #медианное кол-во детей

0.0

In [50]:
#т.к. медианное кол-во детей = 0, то заменим ячейки с отрицательными значениями на 0
df.loc[df['children'] == -1, 'children'] = 0

In [51]:
#проверка
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

<div class="alert alert-block alert-info">
    
столбец `dob_years` 
</div>

In [52]:
#dob_years, обработка нулевого возраста, подсчёт записей с нулевым возрастом
df[df['dob_years'] == 0]['dob_years'].count()

101

In [53]:
#медиана возраста клиентов
dob_years_median = int(df['dob_years'].median())
dob_years_median

42

In [54]:
#замена нулевого возраста на медиану
df.loc[df['dob_years'] == 0, 'dob_years'] = dob_years_median

In [55]:
#проверка
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

<div class="alert alert-block alert-info">
    
столбец `gender` 
</div>

In [56]:
#проверка количества человек со "странным" полом 'XNA'
df[df['gender'] == 'XNA']['gender'].count()

1

In [57]:
#проверка количества человек женского пола
df[df['gender'] == 'F']['gender'].count()

14189

In [58]:
#проверка количества человек мужского пола
df[df['gender'] == 'M']['gender'].count()

7281

In [59]:
#количество женщин примерно в два раза больше чем мужчин.
#Следовательно наиболее вероятно что человек с полом 'XNA'на самом деле имеет женский пол.
#замена на женский пол
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'

In [60]:
#проверка
df['gender'].unique()

array(['F', 'M'], dtype=object)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
### Step 2.5. Formation of additional dataframes of dictionaries, decomposition of the original dataframe.

In [61]:
#новый датафрейм с колонками 'education', 'education_id'
df_education = df[['education', 'education_id']]
df_education.head(3)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1


In [62]:
#новый датафрейм с колонками  family_status family_status_id
df_family_status = df[['family_status', 'family_status_id']]
df_family_status.tail(3)

,family_status,family_status_id
21468,гражданский брак,1
21469,женат / замужем,0
21470,женат / замужем,0


<div class="alert alert-block alert-danger">
    

</div>

In [63]:
#выяснение какой id соответствует семейному статусу
for j in range(0,5):
    
    display(df_family_status[df_family_status['family_status_id'] == j].head(1))

,family_status,family_status_id
0,женат / замужем,0


,family_status,family_status_id
4,гражданский брак,1


,family_status,family_status_id
18,вдовец / вдова,2


,family_status,family_status_id
19,в разводе,3


,family_status,family_status_id
24,Не женат / не замужем,4


<div class="alert alert-block alert-danger">
    

</div>

In [64]:
#удаление из исходного df столбцов 'education' и 'family_status'
df = df.drop(columns=['education', 'family_status'])
df.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью


### Шаг 2.6. Категоризация дохода.

In [65]:
#добавление столбца total_income_category с категориями клиентов по доходам
df.loc[df['total_income'] <= 30000, 'total_income_category'] = 'E'
df.loc[(df['total_income'] >= 30001)&(df['total_income'] <= 50000), 'total_income_category'] = 'D'
df.loc[(df['total_income'] >= 50001)&(df['total_income'] <= 200000), 'total_income_category'] = 'C'
df.loc[(df['total_income'] >= 200001)&(df['total_income'] <= 1000000), 'total_income_category'] = 'B'
df.loc[df['total_income'] >= 1000000, 'total_income_category'] = 'A'

In [66]:
df.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C


### Шаг 2.7. Категоризация целей кредита.
### Step 2.7. Categorization of loan purposes.

In [67]:
#проверка что "категоризация целей кредита" выполнена
df.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C


In [68]:
#проверка
df['purpose_category']

0        операции с недвижимостью
1          операции с автомобилем
2        операции с недвижимостью
3           получение образования
4              проведение свадьбы
                   ...           
21466    операции с недвижимостью
21467      операции с автомобилем
21468    операции с недвижимостью
21469      операции с автомобилем
21470      операции с автомобилем
Name: purpose_category, Length: 21471, dtype: object

### Ответы на вопросы.
### Answers on questions

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [69]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'purpose_category', 'total_income_category'],
      dtype='object')

In [70]:
df.groupby(['children']).agg({'debt':'mean'})

,debt
children,
0,0.075173
1,0.092327
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


In [71]:
children_pivot = df.pivot_table(index=['children'], values='debt', aggfunc=('count', 'mean', 'sum'))

In [72]:
children_pivot

,count,mean,sum
children,,,
0,14154,0.075173,1064
1,4809,0.092327,444
2,2052,0.094542,194
3,330,0.081818,27
4,41,0.097561,4
5,9,0.000000,0
20,76,0.105263,8


##### Вывод 1:

<div class="alert alert-block alert-info">
    
Наблюдается связь, хоть и небольшая, между количеством детей и среднего показателя по возврату кредита. Семьи не имеющие детей возвращают кредиты в срок чаще (7,5% невозвратов), чем семьи имеющие 1,2,3 и 4 ребенка (примерно 9,5% невозвратов). 
    
Семьи имеющие 5 детей вообще ни разу не имели задолженности по кредитам, думаю не стоит этому доверять, т.к. выборка слишком мала, всего для 9ти человек.
    
Семьи имеющие 20 детей имеют максимальный риск невозврата кредита, около 10.5% невозвратов.
</div>

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [73]:
family_status_pivot = df.pivot_table(index=['family_status_id'], values='debt', aggfunc=('count', 'mean'))

In [74]:
family_status_pivot.sort_values('mean')

,count,mean
family_status_id,,
2,959,0.065693
3,1195,0.071130
0,12344,0.075421
1,4163,0.093202
4,2810,0.097509


##### Вывод 2:

<div class="alert alert-block alert-info">
Семьи имеющие 2-й семейный статус (вдова/вдовец) имеют минимальный риск по возврату. Максмимальный риск имеют семьи со статусом 1 и 4.
</div>

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [75]:
income_category_pivot = df.pivot_table(index=['total_income_category'], values='debt', aggfunc=('count', 'mean'))

In [76]:
income_category_pivot.sort_values('mean')

,count,mean
total_income_category,,
D,350,0.060000
B,5041,0.070621
A,25,0.080000
C,16033,0.084825
E,22,0.090909


##### Вывод 3:

<div class="alert alert-block alert-info">
    
Минимальный риск по возврату имеют клиенты с уровнем дохода `D` (около 6%). А максимальный риск клиенты с уровнем дохода `C` и `E` (около 8,7%)
     
</div>

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [77]:
purpose_category_pivot = df.pivot_table(index=['purpose_category'], values='debt', aggfunc=('count', 'mean', 'sum'))

In [78]:
purpose_category_pivot.sort_values('mean')

,count,mean,sum
purpose_category,,,
операции с недвижимостью,10814,0.072314,782
проведение свадьбы,2335,0.079657,186
получение образования,4014,0.092177,370
операции с автомобилем,4308,0.093547,403


##### Вывод 4:

<div class="alert alert-block alert-info">
    
`Операции с недвижимостью` и `проведение свадьбы` находятся в минимальной группе риска (7,2%). 
    
В то время как `получение образования` и `операции с автомобилем` именют выше риск невозврата кредита в срок (около 9,2%).
</div>

##### Вопрос 5:

Как влияют другие данные на возврат кредита в срок?

In [79]:
education_pivot = df.pivot_table(index=['education_id'], values='debt', aggfunc=('count', 'mean'))

In [80]:
education_pivot.sort_values('mean')

,count,mean
education_id,,
4,6,0.000000
0,5251,0.052942
1,15188,0.089808
2,744,0.091398
3,282,0.109929


<div class="alert alert-block alert-info">
    
Наименее рискованные клиенты имеют `высшее образование`.
</div>

In [81]:
gender_pivot = df.pivot_table(index=['gender'], values='debt', aggfunc=('count', 'mean'))

In [82]:
gender_pivot

,count,mean
gender,,
F,14190,0.070049
M,7281,0.102596


<div class="alert alert-block alert-info">

Женщины чаще возвращают кредит в срок, чем мужчины.
</div>

In [83]:
gender_education_pivot = df.pivot_table(index=['gender', 'education_id'], values='debt', aggfunc=('count', 'mean'))

In [84]:
gender_education_pivot.sort_values('mean')

count      mean
gender education_id                 
F      4                 2  0.000000
M      4                 4  0.000000
F      0              3540  0.047175
M      0              1711  0.064874
F      1             10024  0.076816
       2               461  0.088937
M      2               283  0.095406
F      3               163  0.098160
M      1              5164  0.115027
       3               119  0.126050

<div class="alert alert-block alert-info">

Женщины имеющие высшее образование наименее рискованный класс, около 4.7% невозврата.
</div>

##### Вывод 5:

<div class="alert alert-block alert-info">
    
Наименее рискованные клиенты имеют `высшее образование`.
    
Женщины чаще возвращают кредит в срок, чем мужчины.
    
Женщины имеющие высшее образование наименее рискованный класс, около 4.7% невозврата.
</div>

## Общий вывод:
## General conclusion:

<div class="alert alert-block alert-info">

    Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?

Согласно этому исследованию, семейное положение клиента оказывает влияние на возврат кредита в срок.
Семьи имеющие 2-й семейный статус `вдова/вдовец` имеют минимальный риск по возврату (около 6,6%). 
Максмимальный риск имеют семьи со статусом 1 `гражданский брак` и 4 `Не женат/не замужем`, около 9,5%.
    
Количество детей клиента так же оказывает влияние:
Семьи не имеющие детей возвращают кредиты в срок чаще (7,5% невозвратов), чем семьи имеющие 1,2,3 и 4 ребенка(примерно 9,5% невозвратов). 
        
Семьи имеющие 20 детей имеют максимальный риск невозврата кредита, около 10,5%.
   
Минимальный риск по возврату имеют клиенты с уровнем дохода `D` (около 6%). А максимальный риск клиенты с уровнем дохода `C` и `E` (около 8,7%)

`Операции с недвижимостью` и `проведение свадьбы` находятся в минимальной группе риска (7,2%).  
В то время как `получение образования` и `операции с автомобилем` именют выше риск невозврата кредита в срок (около 9,2%).    
Женщины имеющие высшее образование один из наименее рискованный классов, около 4,7% невозвратов.
    

</div>

In [85]:
kredit = [
    
    ['Семьи без детей', 7.5],
    ['Семьи 1,2,3,4 ребенка', 9.5],
    ['Вдова/вдовец', 6.6],
    ['Гражданский брак/Не женат/не замужем', 9.5],
    ['Уровень дохода D', 6],
    ['Уровень дохода C и E', 8.7],
    ['Женщины c высшим образованием ', 4.7],
    ['недвижимость/свадьба', 7.2],
    ['образование/автомобиль', 9.2]
]
col = ['тип клиента', '% невозврата']

In [86]:
table_result = pd.DataFrame(data = kredit, columns=col)

<div class="alert alert-block alert-info">

Результат по исследованию в виде таблицы.
</div>

In [87]:
table_result.sort_values('% невозврата')

,тип клиента,% невозврата
6,Женщины c высшим образованием,4.7
4,Уровень дохода D,6.0
2,Вдова/вдовец,6.6
7,недвижимость/свадьба,7.2
0,Семьи без детей,7.5
5,Уровень дохода C и E,8.7
8,образование/автомобиль,9.2
1,"Семьи 1,2,3,4 ребенка",9.5
3,Гражданский брак/Не женат/не замужем,9.5
